<a href="https://colab.research.google.com/github/kylejohnsonks/Group2/blob/Zack/Load_wine_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres. 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-28 17:02:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.99MB/s    in 0.2s    

2022-03-28 17:02:13 (4.99 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Combined-Weather-Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Load combined weather data into Spark DataFrame
from pyspark import SparkFiles
url = "https://grp2winereviews.s3.amazonaws.com/cleaned_winereviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+----------+----+------------------+------+
|  province|year|           variety|points|
+----------+----+------------------+------+
|   Sicilia|2013|       White Blend|    87|
|    Oregon|2013|        Pinot Gris|    87|
|    Oregon|2012|        Pinot Noir|    87|
|   Sicilia|2013|          Frappato|    87|
|    Alsace|2012|    Gewürztraminer|    87|
|    Alsace|2012|        Pinot Gris|    87|
|California|2011|Cabernet Sauvignon|    87|
|    Alsace|2012|    Gewürztraminer|    87|
|California|2012|Cabernet Sauvignon|    87|
|   Sicilia|2012| Nerello Mascalese|    87|
|California|2012|        Chardonnay|    87|
|    Oregon|2013|        Pinot Noir|    87|
|   Sicilia|2007|       White Blend|    87|
|California|2011|            Merlot|    87|
|   Sicilia|2009|      Nero d'Avola|    87|
|California|2011|        Pinot Noir|    87|
|   Sicilia|2013|       White Blend|    87|
|   Sicilia|2013|      Nero d'Avola|    87|
|   Sicilia|2011|         Red Blend|    87|
|California|2010|      Chenin Bl

In [ ]:
pandas_df=df.toPandas()
variety_counts=pandas_df.variety.value_counts()
replace_varieties=list(variety_counts[variety_counts<2200].index)
replace_varieties

['Pinot Gris',
 'Rosé',
 'Bordeaux-style White Blend',
 'White Blend',
 'Cabernet Franc',
 'Gewürztraminer',
 'Sangiovese Grosso',
 'Petite Sirah',
 'Barbera',
 'Viognier',
 'Rhône-style Red Blend',
 'Corvina, Rondinella, Molinara',
 'Malbec',
 'Grenache',
 'Champagne Blend',
 'Sparkling Blend',
 'Pinot Grigio',
 'Pinot Blanc',
 "Nero d'Avola",
 'Tempranillo',
 'Glera',
 'Garganega',
 'Meritage',
 'Moscato',
 'Rhône-style White Blend',
 'Mourvèdre',
 'Petit Verdot',
 'Vermentino',
 'Dolcetto',
 'G-S-M',
 'Roussanne',
 'Vernaccia',
 'Grillo',
 'Albariño',
 'Grenache Blanc',
 'Chenin Blanc',
 'Primitivo',
 'Grüner Veltliner',
 'Nerello Mascalese',
 'Sémillon',
 'Arneis',
 'Cortese',
 'Muscat',
 'Fumé Blanc',
 'Cabernet Sauvignon-Syrah',
 'Rosato',
 'Alsace white blend',
 'Shiraz',
 'Carmenère',
 'Port',
 'Prugnolo Gentile',
 'Tannat',
 'Sylvaner',
 'Pinot Nero',
 'Marsanne',
 'Sauvignon Blanc-Semillon',
 'Semillon-Sauvignon Blanc',
 'Prosecco',
 'Frappato',
 'Cabernet Sauvignon-Merlot',


In [ ]:
for v in replace_varieties:
  pandas_df.variety=pandas_df.variety.replace(to_replace=replace_varieties,value="Other")
  
pandas_df.variety.value_counts()

Other                       19625
Pinot Noir                  11514
Chardonnay                   9102
Cabernet Sauvignon           7340
Bordeaux-style Red Blend     6337
Red Blend                    5821
Syrah                        3329
Nebbiolo                     2709
Zinfandel                    2693
Sangiovese                   2564
Merlot                       2424
Riesling                     2367
Sauvignon Blanc              2213
Name: variety, dtype: int64

In [ ]:
df=spark.createDataFrame(pandas_df)
df.show()

+----------+----+------------------+------+
|  province|year|           variety|points|
+----------+----+------------------+------+
|   Sicilia|2013|             Other|    87|
|    Oregon|2013|             Other|    87|
|    Oregon|2012|        Pinot Noir|    87|
|   Sicilia|2013|             Other|    87|
|    Alsace|2012|             Other|    87|
|    Alsace|2012|             Other|    87|
|California|2011|Cabernet Sauvignon|    87|
|    Alsace|2012|             Other|    87|
|California|2012|Cabernet Sauvignon|    87|
|   Sicilia|2012|             Other|    87|
|California|2012|        Chardonnay|    87|
|    Oregon|2013|        Pinot Noir|    87|
|   Sicilia|2007|             Other|    87|
|California|2011|            Merlot|    87|
|   Sicilia|2009|             Other|    87|
|California|2011|        Pinot Noir|    87|
|   Sicilia|2013|             Other|    87|
|   Sicilia|2013|             Other|    87|
|   Sicilia|2011|         Red Blend|    87|
|California|2010|             Ot

In [ ]:
# Configure settings for RDS - Group2Wineos password isn't an important password and was kept simple for this exercise.
mode = "append"
jdbc_url="jdbc:postgresql://grp2rdsinstance.cwkbfcctxu7y.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "Group2Wineos", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write df to table in RDS
df.write.jdbc(url=jdbc_url, table='wine_data', mode=mode, properties=config)